In [43]:
import requests
from bs4 import BeautifulSoup

In [44]:
page_numbers_list = []
page_numbers = range(1, 66)
for page in page_numbers:
    href = "https://www.nysenate.gov/search/legislation?sort=last_status_date&searched=true&type=f_bill&bill_session_year=2009&bill_status=SIGNED_BY_GOV&page=" + str(page)
    page_numbers_list = href
#     print(href)

In [45]:
def signed_bills(href):
    raw_html = requests.get(href).content
    soup_doc = BeautifulSoup(raw_html, "html.parser")
    
    all_bills = soup_doc.find_all("div", class_="c-block c-list-item c-block-legislation")

    signed_bills_list_of_dicts = []

    for bill in all_bills:
        bill_number = bill.find("h3", class_="c-bill-num").text.replace("\n", "").replace("Bill ", "").strip()
        bill_link = bill.find("h3", class_="c-bill-num").find("a")['href']
        bill_description = bill.find("p", class_="c-bill-descript").text.replace("\n", "")
        bill_date = bill.find("p", class_="c-bill-update--date").text.replace("\n", "").replace("|  Signed by Governor", "")
        bill_sponsor_tag = bill.find("p", class_="c-bill-update--sponsor")
        if bill_sponsor_tag != None:
            bill_sponsor = bill_sponsor_tag.text.replace("\n", "").replace("Sponsor:", "")
        else:
            bill_sponsor = ""
        signed_bills_list_of_dicts.append(
            {"bill_number": bill_number, "bill_link": "https://www.nysenate.gov" + bill_link, "bill_description": bill_description,
             "bill_date": bill_date, "bill_sponsor": bill_sponsor}
        )
        
    
    return signed_bills_list_of_dicts

In [46]:
page_numbers_main_list = []
page_numbers = range(1, 66)
for page in page_numbers:
    href = "https://www.nysenate.gov/search/legislation?sort=last_status_date&searched=true&type=f_bill&bill_session_year=2009&bill_status=SIGNED_BY_GOV&page=" + str(page)
    bill_list = signed_bills(href)
    
    page_numbers_main_list.extend(bill_list)

In [47]:
import pandas as pd
df_bills = pd.DataFrame(page_numbers_main_list)
pd.set_option("display.max_colwidth", None)
df_bills

,bill_number,bill_link,bill_description,bill_date,bill_sponsor
0,A42007,https://www.nysenate.gov/legislation/bills/2009/a42007/amendment/original,Renames the Brooklyn battery tunnel the Hugh L. Carey tunnel,"December 10, 2010 | Signed by Governor",
1,A42012,https://www.nysenate.gov/legislation/bills/2009/a42012/amendment/original,Updates the state energy conservation construction code to address the requirements of the American Recovery and Reinvestment Act of 2009,"December 10, 2010 | Signed by Governor",
2,A42014,https://www.nysenate.gov/legislation/bills/2009/a42014/amendment/original,Relates to the powers and duties of the dormitory authority of the state of New York relative to the establishment of subsidiaries for certain purposes,"December 10, 2010 | Signed by Governor",
3,S68004,https://www.nysenate.gov/legislation/bills/2009/s68004/amendment/original,"Relates to cost-sharing, deductible or co-insurance for tier IV prescription drugs and unlawful discriminatory practice in prescribing such drugs; repealer","December 10, 2010 | Signed by Governor",
4,S68005,https://www.nysenate.gov/legislation/bills/2009/s68005/amendment/original,Makes technical corrections to provisions of the multiple dwelling law relating to time to obtain certificates of occupancy,"December 10, 2010 | Signed by Governor",
...,...,...,...,...,...
640,A300,https://www.nysenate.gov/legislation/bills/2009/a300/amendment/original,"Enacts the ""New York higher education cares act"" to distribute student residual consumer goods from state universities and colleges to not-for-profit organizations","March 23, 2009 | Signed by Governor",MILLMAN
641,A6740,https://www.nysenate.gov/legislation/bills/2009/a6740/amendment/original,Relates to a special enrollment period,"March 20, 2009 | Signed by Governor",MORELLE
642,A152,https://www.nysenate.gov/legislation/bills/2009/a152/amendment/original,DEBT SERVICE FUND-GENERAL DEBT SERVICE FUND,"March 9, 2009 | Signed by Governor",
643,A4392,https://www.nysenate.gov/legislation/bills/2009/a4392/amendment/original,"Delays from March 1, 2009 until September 1, 2009, the effective date of amendments to the provisions of law providing for powers of attorney","February 25, 2009 | Signed by Governor",WEINSTEIN


In [48]:
def signed_bills_law_section(href):
    raw_html = requests.get(href).content
    soup_doc = BeautifulSoup(raw_html, "html.parser")
    
    law_section_tag = soup_doc.find_all("div", class_="c-block c-bill-section c-bill--details")
    if law_section_tag == []:
        print(href)
        return ""
    law_section = law_section_tag[0].find_all("dd")[1].text
    return law_section

In [49]:
bill_new_numbers = [bill_dict["bill_number"] for bill_dict in page_numbers_main_list]

In [50]:
bill_numbers_main_list = []
for bill in bill_new_numbers:
    href = "https://www.nysenate.gov/legislation/bills/2009/" + bill
    bill_list_sections = signed_bills_law_section(href)
    
    bill_numbers_main_list.append({"bill_number": bill, "bill_section": bill_list_sections})

IndexError: list index out of range

In [ ]:
df_bill_sections = pd.DataFrame(bill_numbers_main_list)
pd.set_option("display.max_colwidth", None)
df_bill_sections

In [ ]:
df = df_bills.join(df_bill_sections.set_index('bill_number'), on='bill_number')

In [ ]:
df